# MPI multiple tasks

Lest start from imports

In [ ]:
import os
from jobqueue_features import (
    MPIEXEC,
    CustomSLURMCluster,
)

from jobqueue_features import on_cluster, mpi_task, get_task_mpi_comm

Let's define task script path.

In [ ]:
script_path = os.path.abspath(
    os.path.join(os.getcwd(), "docker_config", "slurm", "tutorial_tasks", "resources", "helloworld.py")
)

Let's define cluster configuration.

In [ ]:
common_kwargs = {
    "walltime": "00:04:00",
    "cores_per_node": 2,
    "minimum_cores": 2,
    "hyperthreading_factor": 1,
    "ntasks_per_node": 2,
    "memory": "512 MB",
    "mpi_mode": True,
    "env_extra": [
        "export OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1",
        "export OMPI_ALLOW_RUN_AS_ROOT=1",
    ],
    "mpi_launcher": MPIEXEC,
    "local_directory": "/tmp",
    "queue": "batch",
}

Let's start cluster.

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="mpiMultiCluster",
    nodes=1,
    maximum_jobs=2,
    **common_kwargs
)

Let's define task.

In [ ]:
@on_cluster(cluster=custom_cluster)
@mpi_task(cluster_id=custom_cluster.name)
def task(task_name):
    import time
    from mpi4py import MPI

    comm = get_task_mpi_comm()
    size = comm.Get_size()
    name = MPI.Get_processor_name()
    all_nodes = comm.gather(name, root=0)
    if all_nodes:
        all_nodes = list(set(all_nodes))
        all_nodes.sort()
    else:
        all_nodes = []
    # Since it is a return  value it will only get printed by root
    return_string = "Running %d tasks of type %s on nodes %s." % (
        size,
        task_name,
        all_nodes,
    )

    # Add a sleep to make the task substantial enough to require scaling
    time.sleep(1)
    return return_string

Then check the execution of it:

In [ ]:
tasks = []
for x in range(20):
    tasks.append(
        task("task-{}".format(x))
    )
c1_count = 0
c2_count = 0
for job in tasks:
    result = job.result()
    if 'c1' in result:
        c1_count += 1
    elif 'c2' in result:
        c2_count += 1
print("c1: {} \nc2: {}".format(c1_count, c2_count))

It's alive!